In [1]:
using Pkg
Pkg.add("Latexify");
Pkg.add("LaTeXStrings");

   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [15]:
println("Homework 1 - Jacob Lorenz")

Homework 1 - Jacob Lorenz


# Question 1 - Warm-up


$$
    \text{max} \; 5x_1 - x_2 + 15x_3 \\
    \text{subject to:} \\
    2x_1 \geq x_2 + x_3 \\
    0 \leq x_j \leq 3, j \in {1,2,3} \\
$$

**(a) - Write Julia code to solve this problem using Clp, ECOS, and SCS Solvers. Do all give the same result?**

In [14]:
using Pkg
Pkg.add("JuMP")
Pkg.add("Clp")
Pkg.add("ECOS")
Pkg.add("SCS")
using JuMP, Clp, ECOS, SCS, Latexify, LaTeXStrings

A = [-2 1 1; 1 0 0; 0 1 0; 0 0 1]
b = [0,3,3,3]
c = [5,-1,15]
x = [1,2,3]

m1 = Model()

@variable(m1, x[x] >= 0)
for i in 1:size(A, 1)
    @constraint(m1, sum(A[i,j] * x[j] for j in 1:size(A,2)) <= b[i])
end
@objective(m1, Max, sum(c[j] * x[j] for j in 1:size(c,1)))


println("Stats for the Clp Solver")
println("===============================")
set_optimizer(m1, Clp.Optimizer)
@time optimize!(m1)

for i in 1:size(x,1)
    println("Value for x", i, ": ", value(x[i]), ".")
end
println()

println("Stats for the ECOS Solver")
println("===============================")
set_optimizer(m1, ECOS.Optimizer)
@time optimize!(m1)

for i in 1:size(x,1)
    println("Value for x", i, ": ", value(x[i]), ".")
end
println()

println("Stats for the SCS Solver")
println("===============================")
set_optimizer(m1, SCS.Optimizer)
@time optimize!(m1)

for i in 1:size(x,1)
    println("Value for x", i, ": ", value(x[i]), ".")
end


  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...


Stats for the Clp Solver
  0.000690 seconds (1.46 k allocations: 119.547 KiB)
Value for x1: 3.0.
Value for x2: 0.0.
Value for x3: 3.0.

Stats for the ECOS Solver
  0.002329 seconds (2.52 k allocations: 194.297 KiB)
Value for x

No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


1: 2.9999999997920908.
Value for x2: 1.2382457716351817e-10.
Value for x3: 3.000000000021709.

Stats for the SCS Solver
  0.002444 seconds (1.61 k allocations: 122.703 KiB)
Value for x1: 2.999999937039052.
Value for x2: 9.449801729506784e-7.
Value for x3: 3.0000008986196134.
Coin0506I Presolve 0 (-4) rows, 0 (-3) columns and 0 (-6) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 60
Coin0511I After Postsolve, objective 60, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 60 - 0 iterations time 0.002, Presolve 0.00

ECOS 2.0.5 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -2.850e+01  -1.078e+02  +1e+02  2e-01  3e-01  1e+00  2e+01    ---    ---    1  1  - |  -  - 
 1  -5.850e+01  -7.123e+01  +3e+01  2e-02  5e-02  9e-01  4e+00  0.8418  6e-02   0  0  0 |  0  0
 2  -5.858e+01  -6.145e+01  

By and large, all of them give the same result. Optimal $x_1 = 3, x_2 = 0, x_3 = 3$. Upon visual inspection of the objective function, this result makes sense. The objective rewards $x_1$ and $x_3$ and punishes $x_2$, so it only seems natural that $x_1$ and $x_3$ are the highest value allowed by the conditions, and nothing is allotted to $x_2$. Only differences between the 3 solvers seems to be the condition of each solving treating the numbers as integers versus real numbers.

**(b) - Which solver is fastest (@time macro before optimze!())? Can you say why?**

From the stats listed above, we see the following times for each solver:
- Clp: 0.000690 seconds (1.46 k allocations: 119.547 KiB)
- ECOS: 0.002329 seconds (2.52 k allocations: 194.297 KiB)
- SCS: 0.002444 seconds (1.61 k allocations: 122.703 KiB)

With these results, we conclude that the Clp solver was the fastest by almost an order of magnitude! I do not **know** why this one would be faster, but from my observations of the optimal answers, I would guess that ECOS and SCS are slower due to them handling real numbers instead of integers. The shortcuts that can be taken by Clp due to considering the variables as integers surely cuts out some of the processing time needed by the other two, so much so that it was almost 10x faster.

**(c) Consider changing the first contraint to $2x_1 \geq x_2 + x_3 + \alpha$ where $\alpha$ is some real number. For all values of $\alpha$ larger than a certain threshold value, the problem will become infeasible. (In Julia, the solver will terminate with a status INFEASIBLE). What is this threshold value of $\alpha$? Can you explain by examining the constraints why this value makes the problem infeasible?**

The threshold value for $\alpha$ is 6. That is, for any $\alpha \gt 6$, the LP becomes infeasible. The reasoning based on the constraints is this:

1) We notice that the negative contributor to this constraint is $x_2$ with a factor of 2.

2) We also notice that as $\alpha$ increases, the constaint ($-\alpha$) reduces.

3) We notice that all three variables, $x_1, x_2,$ and $x_3$ are all between 0 and 3.

4) We also recognize that the edge case to find our threshold comes when $x_1 = 0$ and $x_3 = 0$, and when $x_2 = 3$.

5) Therefore, the minimum quantity that can be obtained in this restriction is -6, so having a constraint of -7 or less will cause the LP to become infeasible, meaning that the threshold value is $\alpha = 6$.

# Question 2 - Standard Form

**Recall that in class we defined the "standard form" of a linear program to be**

$$
    \text{max}\: c^tx \: \text{subject to} Ax \leq b, x \geq 0
$$
        

**Consider the following linear program which is not in standard form:**

$$
    \text{min} \; 3z_1 + 2z_2 - z_3 \\
    \text{subject to:} \\
    2z_1 - z_2 + 3z_3 = 10 \\
    0 \leq z_2 \leq 5 \\
    -5 \leq z_3 \leq 20
$$
        
**(a) Transform the given LP into our standard form. How are the components of x related to the components of $z = (z_1,z_2,z_3)$? Write out explicitely the quantities A, b, c.**

I will use a number of techniques from the lectures on Linear Programming and transform the original problem to standard form in a number of steps.

1) Create a new variable for $z_3$
Since $z_3$ is bounded, we want to convert it to a non-negative variable instead. To do this, we let $x_1 = z_3 + 5$. Furthermore, we can now let $z_3 = x_1 - 5$, where $x_1 \geq 0$ and $x_1 \leq 25$.

2) Notice that $z_1$ is unbounded
Since $z_1$ is unbounded, we can instead represent $z_1$ as the difference of two nonnegative numbers. Let $z_1 = x_3 - x_4$ where $x_3,x_4 \geq 0$

3) We let $z_2$ remain unchanged; however, for consistency, we will let $z_2 = x_2$

We are not quite finished with the transformation yet; however, with the variable replacements definedd, now might be a good time to reorganize our original equation with the transformations up to this point applied. With the transformations thus far, we now have:

$$
    \text{min} \; -3x_4 + 3x_3 + 2x_2 - x_1 + 5 \\
    \text{subject to:} \\
    2x_3 - 2x_4 - x_2 + 3x_1 = 25 \\
    0 \leq x_1 \leq 25 \\
    0 \leq x_2 \leq 5 \\
    x_3, x_4 \geq 0
$$

4) We now proceed with the penultimate transformation step. We notice that the first constraint is an equality, we would like to transform this into a pair of complementary inequalities.
Instead of the equality, we can transform the first constraint, after a litle manipulation, into:

$$
    2x_3 - 2x_4 - x_2 + 3x_1 \leq 25, \\
    2x_4 - 2x_3 + x_2 - 3x_1 \leq -25
$$

5) Finally, the last step we need to transform into standard form is to convert the objective from a minimization to a maximization problem.
For this, we use the general conversion:

$$
    min( f(x) ) = -max( -f(x) )
$$

Giving the following new objective function:

$$
    -max(3x_4 - 3x_3 - 2x_2 + x_1 - 5)
$$

With these 5 steps applied above, the LP is transformed into standard form. We now have the following system:

$$
    z_1 = x_3 - x_4, \;\; x_3,x_4 \geq 0 \\
    z_2 = x_2 \\
    z_3 = x_1 - 5
$$

And we get the following quantities for A, b, c, and x of the transformed, standard form LP:

$$
    \text{A} = 
    \left[ \begin{array}{rl}
                3 \\
                -3 \\
                1 \\
                0 \\
            \end{array} \right.
            \left. \begin{array}{rl}
                -1 \\
                1 \\
                0 \\
                1 \\
            \end{array} \right. 
            \left. \begin{array}{rl}
                2 \\
                -2 \\
                0 \\
                0 \\
            \end{array} \right. 
            \left. \begin{array}{rl}
                -2 \\
                2 \\
                0 \\
                0 \\
            \end{array}\right], \;\;\;
    \text{b} = 
    \left[ \begin{array}{rl}
                25 \\
                -25 \\
                25 \\
                5 \\
            \end{array}\right], \;\;\;
    \text{c} = 
    \left[ \begin{array}{rl}
                1 \\
                -2 \\
                -2 \\
                3 \\
            \end{array}\right], \;\;\;
    \text{x} = 
    \left[ \begin{array}{rl}
                x_1 \\
                x_2 \\
                x_3 \\
                x_4 \\
            \end{array}\right]  
$$

**(b) Solve both versions of the LP using JuMP and show that you can recover the optimal z and objective value by solving your transformed version of the LP.**

In [19]:
A = [3 -1 2 -2; -3 1 -2 2; 1 0 0 0; 0 1 0 0]
b = [25,-25,25,5]
c = [1,-2,-3,3]
x = [1,2,3,4]

m1 = Model()

@variable(m1, x[x] >= 0)
for i in 1:size(A, 1)
    @constraint(m1, sum(A[i,j] * x[j] for j in 1:size(A,2)) <= b[i])
end
@objective(m1, Max, sum(c[j] * x[j] for j in 1:size(c,1)))


println("Stats for the Clp Solver")
println("===============================")
set_optimizer(m1, Clp.Optimizer)
@time optimize!(m1)

for i in 1:size(x,1)
    println("Value for x", i, ": ", value(x[i]), ".")
end
println()

Stats for the Clp Solver
  0.000664 seconds (1.46 k allocations: 119.938 KiB)
Value for x1: 25.0.
Value for x2: 0.0.
Value for x3: 1.4999999975e10.
Value for x4: 1.5e10.

Coin0506I Presolve 2 (-2) rows, 4 (0) columns and 8 (-2) elements
Clp0006I 0  Obj -0 Primal inf 8.3333323 (1) Dual inf 5.499998 (2)
Clp0006I 1  Obj 100
Clp0000I Optimal - objective value 100
Coin0511I After Postsolve, objective 100, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 100 - 1 iterations time 0.002, Presolve 0.00


In [12]:
c = [3,2,-1]
x = [1,2,3]

m1 = Model()

@variable(m1, z[x])
@constraint(m1, 0 <= z[2] <= 5)
@constraint(m1, -5 <= z[3] <= 20)
@constraint(m1, 2z[1] - z[2] + 3z[3] == 10)
@objective(m1, Min, sum(c[j] * z[j] for j in 1:size(c,1)))


println("Stats for the Clp Solver")
println("===============================")
set_optimizer(m1, Clp.Optimizer)
@time optimize!(m1)

for i in 1:size(z,1)
    println("Value for z", i, ": ", value(z[i]), ".")
end
println()

Stats for the Clp Solver
  0.000681 seconds (1.54 k allocations: 126.969 KiB)
Value for z1: -25.0.
Value for z2: 0.0.
Value for z3: 20.0.

Coin0506I Presolve 0 (-3) rows, 0 (-3) columns and 0 (-5) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value -95
Coin0511I After Postsolve, objective -95, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -95 - 0 iterations time 0.002, Presolve 0.00


From the results above, we can see the following:

In the original LP without any standard form refactoring, we see that the optimal values were determined to be

$$
    z_1 = -25 \\
    z_2 = 0 \\
    z_3 = 20 \\
    Objective = -95
$$

From the transformed, standard form LP, we see that the optimal values were determined to be

$$
    x_1 = 25 \\
    x_2 = 0 \\
    x_3 = 1.999999975e^{10} \\
    x_4 = 1.5e^{10} \\
    Objective = -100
$$

One important thing to note; however, is the objective for the standard transformed LP is actually not -100. We excluded the term of -5, which takes effect before negating the 100 for the conversion from min to max. So in reality, the objective = -(100-5) = -95.

Now that we have all of our raw variables, let's try to find how the answers to the original can be recovered from the variables of the transformed. In the transformation process, we made the following substitutions:

$$
    z_1 = x_3 - x_4 = 1.999999975e^{10} - 1.5e^{10} = 1999999975 - 1500000000 = -25 \\
    z_2 = x_2 = 0 \\
    z_3 = x_1 - 5 = 25 -5 = 20
$$

As we can see, the transformation from the standard variables back to the equivalent original z variables leads to the exact same values for the z variables, and thus will result in recovering the same optimal objective value of -95.

# Question 3 - Polyhedron Modeling

**Find A and b such that $Ax \leq b$ describes the diamond-shaped polyhedron in three dimensions, that is defined by the following six vertices:
(1,1,0), (1,-1,0), (-1,-1,0), (-1,1,0), (0,0,1), (0,0,-1)**

First thing we note, is that the shape produced by connecting the vertex on the outer edges is that of a octahedron, or 8-faced diamond, shape. This is important to note because our polyhedron is defined by the combination of its planes that make up its edges. With that being said, it would be prudent to focus on each face individually in the hope that when all is said and done, we have the defining equation of the polyhedron.

First, I split the shape into 8-faces based on the logical triangular neighboring vertices. Let the first vertex be X, the 2nd vertex be Y, and the 3rd vertex be Z for each of the following sets of vertices:

Face 1: (1,1,0), (1,-1,0), (0,0,1) 

Face 2: (1,1,0), (1,-1,0), (0,0,-1)

Face 3: (1,1,0), (-1,1,0), (0,0,1)

Face 4: (1,1,0), (-1,1,0), (0,0,-1)

Face 5: (-1,1,0), (-1,-1,0), (0,0,1)

Face 6: (-1,1,0), (-1,-1,0), (0,0,-1)

Face 7: (-1,-1,0), (1,-1,0), (0,0,1)

Face 8: (-1,-1,0), (1,-1,0), (0,0,-1)

It is also important to note alignment. Not all of the planes are above the points we want, some are below.

After we have all the relevant planes of our polyhedron, we use some results from linear algebra to find the equation of the planes, and hence the polyhedron. Namely, that the cross product of two vectors will be orthogonal to those vectors, and the normal vector of a plane is what's used to understand the plane's equation. 

So, we next find two vectors for each face. To do this is easy. Since we have 3 vertices which define each plane, we know each vertex lies in the plane, and hence, vectors connecting two of the three points would  certainly lie within the plane. Thus, for each face, we define two vectors XY and XZ:

Face 1: XY = (0,-2,0), XZ = (-1,-1,1)

Face 2: XY = (0,-2,0), XZ = (-1,-1,-1)

Face 3: XY = (-2,0,0), XZ = (-1,-1,1)

Face 4: XY = (-2,0,0), XZ = (-1,-1,-1)

Face 5: XY = (0,-2,0), XZ = (1,-1,1)

Face 6: XY = (0,-2,0), XZ = (1,-1,-1)

Face 7: XY = (2,0,0), XZ = (1,1,1)

Face 8: XY = (2,0,0), XZ = (1,1,-1)

Now that we have 2 vectors that we know lie within each respective plane, we can compute the normal vector to each plane by taking the cross product of each pair of XY and XZ. After taking the cross product, you can then use the orthogonal vector and the intercepts of one of the original vertices of the plane to derive an equation for each face's plane. The equations are found below:

Face 1: -X - Z = -1

Face 2: X - Z = 1

Face 3: Z + Y = 1

Face 4: Z - Y = -1

Face 5: Z - X = 1

Face 6: X + Z = -1

Face 7: Z - Y = 1

Face 8: Z + Y = -1

We now have 8 equations describing the outer boundaries of the polyhedron, and thus, the polyhedron's outline. We; however, want the polyhedron as a solid, and thus convert all the hyperplane equations into halfspace equations by replacing the equality with an inequality. Although we eventually want all inequalities to be less than or equals, it's important to understand which way the inequality goes on a face by face basis. For this, we notice that the point (0,0,0) is on the correct side of each face of the polyhedron as it resides in the center. So we can simply use (0,0,0) to determine whether we want the greater or lesser halfspace of each face's plane. The results of that are:

Face 1: $-X - Z \geq -1$

Face 2: $X - Z \leq 1$

Face 3: $Z + Y \leq 1$

Face 4: $Z - Y \geq -1$

Face 5: $Z - X \leq 1$

Face 6: $X + Z \geq -1$

Face 7: $Z - Y \leq 1$

Face 8: $Z + Y \geq -1$

Furthermore, for each instance of $\geq$, we need to manipulate the expression to flip the sign. Generally, $Ax \leq b$ ---> $(-A)x \geq (-b)$. Therefore, we get the following after reversing the sign on Faces 1,4,6,8:

Face 1: $X + Z \leq 1$

Face 2: $X - Z \leq 1$

Face 3: $Y + Z \leq 1$

Face 4: $Y - Z \leq 1$

Face 5: $Z - X \leq 1$

Face 6: $-Z - X \leq 1$

Face 7: $Z - Y \leq 1$

Face 8: $-Z - Y \leq 1$

With the final 8 equations describing the halfspaces of our polyhedron, we can transform them into $Ax \leq b$ form, giving us the following results for A and b:

$$
    \text{A} = 
    \left[ \begin{array}{rl}
                1 \\
                1 \\
                0 \\
                0 \\
                -1 \\
                -1 \\
                0 \\
                0 \\
            \end{array} \right.
            \left. \begin{array}{rl}
                0 \\
                0 \\
                1 \\
                1 \\
                0 \\
                0 \\
                -1 \\
                -1 \\
            \end{array} \right. 
            \left. \begin{array}{rl}
                1 \\
                -1 \\
                1 \\
                -1 \\
                1 \\
                -1 \\
                1 \\
                -1 \\
            \end{array}\right], \;\;\;
    \text{b} = 
    \left[ \begin{array}{rl}
                1 \\
                1 \\
                1 \\
                1 \\
                1 \\
                1 \\
                1 \\
                1 \\
            \end{array}\right]
$$

# Problem 4: Manufacturing Transistors

**Silicon Valley Corp (Solvco) manufactures transistors. An important aspect of the manufacturing process is the melting of germanium in a furnace. Unfortunately, the melting process yields germanium of highly variable quality. The results of the process can be classified into five grdes: A, B, C, D, Defective. Two methods can be used for the melting. Method 1 costs $50$ per transistor, and Method 2 costs $70$ per transistor. The following distribution describes the quality of germanium  you can expect from using each method:.**

**Method 1:**

- defective: 30%
- D: 30%
- C: 20%
- B: 15%
- A: 5%

**Method 2:**

- defective: 20%
- D: 20%
- C: 25%
- B: 20%
- A: 15%

**Silvco has sufficient capacity to melt germanium for at most 20,000 transistors per month. Silvco's monthly demands are for 1000 Grade A, 2000 Grade B, 3000 Grade C, and 3000 Grade D transistors.**

**(a) Determine how Silvco can meet its demands and satisfy all constraints at minimal cost.**

To begin solving this problem, we first determine the decision variables and place constraints, as close to standard form as we can. When thinking about the points in this model where we have a choice, it comes down to deciding how many units of germanium to melt for both methods, $M_1$ and $M_2$. We then notice that all intake of particular grades of transistors comes down to linear combinations of $M_1$ and $M_2$, which we can manipulate a little bit to get the entire model into standard LP form. Since the original connstraints for transistor quality are $\geq$, we will manipulate the expression to obtain an equivalent $\leq$. Specifically:

$$
    \text{min} \; 50M_1 + 70M_2 \\
    \text{subject to:} \\
    M_1 + M_2 \leq 20000 \\
    0 \leq M_1 \leq 20000 \\
    0 \leq M_2 \leq 20000 \\
    -.05M_1 - .15M_2 \leq -1000 \\
    -.15M_1 - .20M_2 \leq -2000 \\
    -.20M_1 - .25M_2 \leq -3000 \\
    -.30M_1 - .20M_2 \leq -3000
$$

To summarize our LP for part (a), we have:

$$
    \text{A} = 
    \left[ \begin{array}{rl}
                1 \\
                1 \\
                0 \\
                -.05 \\
                -.15 \\
                -.20 \\
                -.30 \\
            \end{array} \right. 
            \left. \begin{array}{rl}
                1 \\
                0 \\
                1 \\
                -.15 \\
                -.20 \\
                -.25 \\
                -.20 \\
            \end{array}\right], \;\;\;
    \text{b} = 
    \left[ \begin{array}{rl}
                20000 \\
                20000 \\
                20000 \\
                -1000 \\
                -2000 \\
                -3000 \\
                -3000 \\
            \end{array}\right], \;\;\;
    \text{c} = 
    \left[ \begin{array}{rl}
                -50 \\
                -70 \\
            \end{array}\right], \;\;\;
    \text{x} = 
    \left[ \begin{array}{rl}
                M_1 \\
                M_2 \\
            \end{array}\right]  
$$

With the understanding that our answer for the original LP is now the -max since we converted the minimization to a maximization for standard form.

In [7]:
A = [1 1; 1 0; 0 1; -.05 -.15; -.15 -.20; -.20 -.25; -.30 -.20]
b = [20000,20000,20000,-1000,-2000,-3000,-3000]
c = [-50,-70]
x = [1,2]

m1 = Model()

@variable(m1, x[x] >= 0)
for i in 1:size(A, 1)
    @constraint(m1, sum(A[i,j] * x[j] for j in 1:size(A,2)) <= b[i])
end
@objective(m1, Max, sum(c[j] * x[j] for j in 1:size(c,1)))


println("Stats for the Clp Solver")
println("===============================")
set_optimizer(m1, Clp.Optimizer)
@time optimize!(m1)

for i in 1:size(x,1)
    println("Value for x", i, ": ", value(x[i]), ".")
end
println()

Stats for the Clp Solver
  0.000666 seconds (1.48 k allocations: 121.719 KiB)
Value for x1: 11428.571428571428.
Value for x2: 2857.142857142858.

Coin0506I Presolve 5 (-2) rows, 2 (0) columns and 10 (-2) elements
Clp0006I 0  Obj -0 Primal inf 38666.667 (4)
Clp0006I 2  Obj -771428.57
Clp0000I Optimal - objective value -771428.57
Coin0511I After Postsolve, objective -771428.57, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -771428.5714 - 2 iterations time 0.002, Presolve 0.00


Given the results of the Clp solver running on our LP above, we see that the optimal number of times to use $M_1 = 11,428.57$, and the optimal number of times to use $M_2 = 2,857.14$.

With these stats, our total cost comes to 771,428.57

**(b) We take the same setup as above and add a level of complication to it. Assume now that after the initial melting stage described above, Silvco can refire melted germanium once in an attempt to improve its quality. It costs $25$ to refire the melted germanium for one transistor. The results of the refiring process are shown below. For example, if Grade B germanium is refired, 50% of the resulting Germanium will still be grade B, and 50% will be grade A:**

**Refiring:**

- defective: def-30, D-25, C-15, B-20, A-10
- D: D-30, C-30, B-20, A-20
- C: C-40, B-30, A-30
- B: B-50, A-50

**Silvco has sufficient furnace capacity to melt or refire germanium for at most 20,000 transistors per month with the same monthly demands. Model the melting / refiring process and determine the minimum cost of germanium processing required to meet the demands.**

To add onto part (a), this problem adds a couple different constraints. The first thing worth mentioning is that I chose to use two separate variables to model the refiring process, $R_1$ and $R_2$, which are the secondary step option of $M_1$ and $M_2$, respectively. I chose this because I felt it was important to differentiate which method the refired transistor came from originally since the original two methods each have different grade distributions. We can then once again represent the quantity of each transistor grade using the 4 mentioned decision variables. The coefficients for each $R_x$ are derived from the linear combination of likelihood of a particular grade given the method used, multiplied by the likelihood of becoming the target grade after refire. With that being said, we'll begin the model:

$$
    \text{min} \; 50M_1 + 70M_2 + 25(R_1 + R_2) \\
    \text{subject to:} \\
    M_1 + M_2 + R_1 + R_2 \leq 20000 \\
    R_1 - M_1 \leq 0 \\
    R_2 - M_2 \leq 0 \\
    -.05M_1 - .15M_2 - .175R_1 - .085R_2 \leq -1000 \\
    -.15M_1 - .20M_2 - .105R_1 - .055R_2 \leq -2000 \\
    -.20M_1 - .25M_2 - .015R_1 + .03R_2 \leq -3000 \\
    -.3M_1 - .20M_2 + .135R_1 + .14R_2 \leq -3000 \\
    M_1, M_2, R_1, R_2 \geq 0 \\
$$

The final thing we will do before solving the LP is changing the objective from a minimization to a maximization, so that our previously created generic solver will work. Giving:

$$
    \text{-max} \; -50M_1 - 70M_2 - 25(R_1 + R_2) \\
    \text{subject to:} \\
    M_1 + M_2 + R_1 + R_2 \leq 20000 \\
    R_1 - M_1 \leq 0 \\
    R_2 - M_2 \leq 0 \\
    -.05M_1 - .15M_2 - .175R_1 - .085R_2 \leq -1000 \\
    -.15M_1 - .20M_2 - .105R_1 - .055R_2 \leq -2000 \\
    -.20M_1 - .25M_2 - .015R_1 + .03R_2 \leq -3000 \\
    -.3M_1 - .20M_2 + .135R_1 + .14R_2 \leq -3000 \\
    M_1, M_2, R_1, R_2 \geq 0 \\
$$

To summarize our LP for part (b), we have:

$$
    \text{A} = 
    \left[ \begin{array}{rl}
                1 \\
                -1 \\
                0 \\
                -.05 \\
                -.15 \\
                -.20 \\
                -.30 \\
            \end{array} \right.
            \left. \begin{array}{rl}
                1 \\
                0 \\
                -1 \\
                -.15 \\
                -.20 \\
                -.25 \\
                -.20 \\
            \end{array} \right. 
            \left. \begin{array}{rl}
                1 \\
                1 \\
                0 \\
                -.175 \\
                -.105 \\
                -.015 \\
                .135 \\
            \end{array} \right.
            \left. \begin{array}{rl}
                1 \\
                0 \\
                1 \\
                -.085 \\
                -.055 \\
                .03 \\
                .14 \\
            \end{array}\right], \;\;\;
    \text{b} = 
    \left[ \begin{array}{rl}
                20000 \\
                0 \\
                0 \\
                -1000 \\
                -2000 \\
                -3000 \\
                -3000 \\
            \end{array}\right], \;\;\;
    \text{c} = 
    \left[ \begin{array}{rl}
                -50 \\
                -70 \\
                -25 \\
                -25 \\
            \end{array}\right], \;\;\;
    \text{x} = 
    \left[ \begin{array}{rl}
                M_1 \\
                M_2 \\
                R_1 \\
                R_2 \\
            \end{array}\right]
$$

With the understanding that our answer for the original LP is now the -max since we converted the minimization to a maximization for standard form.

In [8]:
A = [1 1 1 1; 
    -1 0 1 0; 
    0 -1 0 1; 
    -.05 -.15 -.175 -.085; 
    -.15 -.20 -.105 -.055;
    -.20 -.25 -.015 .03;
    -.30 -.20 .135 .14]
b = [20000,0,0,-1000,-2000,-3000,-3000]
c = [-50,-70,-25,-25]
x = [1,2,3,4]

m1 = Model()

@variable(m1, x[x] >= 0)
for i in 1:size(A, 1)
    @constraint(m1, sum(A[i,j] * x[j] for j in 1:size(A,2)) <= b[i])
end
@objective(m1, Max, sum(c[j] * x[j] for j in 1:size(c,1)))


println("Stats for the Clp Solver")
println("===============================")
set_optimizer(m1, Clp.Optimizer)
@time optimize!(m1)

for i in 1:size(x,1)
    println("Value for x", i, ": ", value(x[i]), ".")
end
println()

Stats for the Clp Solver
  0.000653 seconds (1.48 k allocations: 122.938 KiB)
Value for x1: 11428.571428571428.
Value for x2: 2857.1428571428573.
Value for x3: 0.0.
Value for x4: 0.0.

Coin0506I Presolve 7 (0) rows, 4 (0) columns and 24 (0) elements
Clp0006I 0  Obj 0 Primal inf 100006.71 (4)
Clp0006I 2  Obj -771428.57
Clp0000I Optimal - objective value -771428.57
Clp0032I Optimal objective -771428.5714 - 2 iterations time 0.002


Interestingly enough, even with the added options of refiring the transistors, the LP above actually chose to not refire any transistors and instead resulted in the exact same decisions as in part (a), that is, melting 11,428.57 units of germanium with Method 1, and melting 2,857.14 units of germanium with Method 2. Namely,

$$
    M_1 = 11,428.57 \\
    M_2 = 2,857.14 \\
    Objective = 771,428.57
$$

In conclusion, this result makes me a little suspicious that I made a mistake; however, I went through my work and reasoning and I can't find any mistakes. I think my decision to split the refiring out into two individual variables is logical and correct because you need to know which method originally the transistors came from to determine the final distribution for transistor grade after hypothetical refiring. If I was to lump everything for refiring into one decision variable, we would lose some information and the final amounts would be skewed. I will accept my result and trust the process, although I am skeptical.